뉴스 기사 제목 데이터에 대한 이해
약 15년 동안 발행되었던 뉴스 기사 제목을 모아놓은 영어 데이터를 아래 링크에서 다운받을 수 있습니다.

데이터 다운 받으면 됨

링크 : https://www.kaggle.com/therohk/million-headlines

In [ ]:
import pandas as pd
import urllib.request
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
data = pd.read_csv('cnbc.csv', error_bad_lines=False)
print('뉴스 제목 개수 :',len(data))

뉴스 제목 개수 : 2800


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data.head(5)

,Headlines,Date,Description
0,Cramer: Never buy a stock all at once - you'll...,2017/12/22,Jim Cramer doubled down on his key investing r...
1,Cramer: I helped investors through the 2010 fl...,2017/12/22,"Jim Cramer built on his ""nobody ever made a di..."
2,Cramer says owning too many stocks and too lit...,2017/12/22,Jim Cramer broke down why owning fewer stocks ...
3,Markets lack Christmas cheer,2017/12/26,"According to Kensho, here's how markets have f..."
4,S&P tends to start new year bullish after this...,2017/12/27,The S&P is on track to end the year up 20 perc...


기사의 헤드라인만 필요하기 때문에 text라는 이름으로 따로 저장

In [ ]:
text = data[['Headlines']]
text

,Headlines
0,Cramer: Never buy a stock all at once - you'll...
1,Cramer: I helped investors through the 2010 fl...
2,Cramer says owning too many stocks and too lit...
3,Markets lack Christmas cheer
4,S&P tends to start new year bullish after this...
...,...
2795,Cramer's lightning round: I would just stay lo...
2796,IQ Capital CEO Keith Bliss says tech and healt...
2797,"Cramer's week ahead: Big week for earnings, ev..."
2798,Cramer's lightning round: I would own Teradyne


## 텍스트 전처리
불용어 제거, 표제어 추출, 길이가 짧은 단어 제거라는 세 가지 전처리 기법
NLTK의 word_tokenize를 통해 단어 토큰화를 수행합니다.

토큰화 링크
https://www.nltk.org/api/nltk.tokenize.html

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
text['Headlines'] = text.apply(lambda row: nltk.word_tokenize(row['Headlines']), axis=1)

<ipython-input-6-2662cf5f5ceb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['Headlines'] = text.apply(lambda row: nltk.word_tokenize(row['Headlines']), axis=1)


In [ ]:
text['Headlines']

0       [Cramer, :, Never, buy, stock, -, 'll, almost,...
1       [Cramer, :, I, help, investors, 2010, flash, c...
2       [Cramer, say, own, many, stock, little, cash, ...
3                       [Markets, lack, Christmas, cheer]
4       [S, &, P, tend, start, new, year, bullish, hap...
                              ...                        
2795    [Cramer, 's, lightning, round, :, I, would, st...
2796    [IQ, Capital, CEO, Keith, Bliss, say, tech, he...
2797    [Cramer, 's, week, ahead, :, Big, week, earn, ...
2798    [Cramer, 's, lightning, round, :, I, would, Te...
2799    [Jim, Cramer, :, A, better, way, invest, Covid...
Name: Headlines, Length: 2800, dtype: object

As dealing with multi-word tokenization, another way would be to retokenize the extracted tokens with NLTK Multi-Word Expression tokenizer:

In [ ]:
mwtokenizer = nltk.MWETokenizer(separator='')
mwtokenizer.add_mwe(('S', '&', 'P'))
text['Headlines'] = text.apply(lambda row: mwtokenizer.tokenize(row['Headlines']), axis=1)

<ipython-input-18-743175495e7b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['Headlines'] = text.apply(lambda row: mwtokenizer.tokenize(row['Headlines']), axis=1)


In [ ]:
text['Headlines']

0       [Cramer, :, Never, buy, stock, -, 'll, almost,...
1       [Cramer, :, I, help, investors, 2010, flash, c...
2       [Cramer, say, own, many, stock, little, cash, ...
3                       [Markets, lack, Christmas, cheer]
4          [S&P, tend, start, new, year, bullish, happen]
                              ...                        
2795    [Cramer, 's, lightning, round, :, I, would, st...
2796    [IQ, Capital, CEO, Keith, Bliss, say, tech, he...
2797    [Cramer, 's, week, ahead, :, Big, week, earn, ...
2798    [Cramer, 's, lightning, round, :, I, would, Te...
2799    [Jim, Cramer, :, A, better, way, invest, Covid...
Name: Headlines, Length: 2800, dtype: object

In [ ]:
text

,Headlines
0,"[Cramer, :, Never, buy, stock, -, 'll, almost,..."
1,"[Cramer, :, I, help, investors, 2010, flash, c..."
2,"[Cramer, say, own, many, stock, little, cash, ..."
3,"[Markets, lack, Christmas, cheer]"
4,"[S&P, tend, start, new, year, bullish, happen]"
...,...
2795,"[Cramer, 's, lightning, round, :, I, would, st..."
2796,"[IQ, Capital, CEO, Keith, Bliss, say, tech, he..."
2797,"[Cramer, 's, week, ahead, :, Big, week, earn, ..."
2798,"[Cramer, 's, lightning, round, :, I, would, Te..."


###stopword
NLTK가 제공하는 영어 불용어를 통해서 text 데이터로부터 불용어를 제거

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
text['Headlines'] = text['Headlines'].apply(lambda x: [word for word in x if word not in (stop_words)])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-23-85228ec1c0d3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['Headlines'] = text['Headlines'].apply(lambda x: [word for word in x if word not in (stop_words)])


In [ ]:
text

,Headlines
0,Cramer Never buy stock 'll almost definitely g...
1,Cramer help investors 2010 flash crash follow ...
2,Cramer say many stock little cash set failure
3,Markets lack Christmas cheer
4,S&P tend start new year bullish happen
...,...
2795,Cramer lightning round would stay long Wex
2796,Capital CEO Keith Bliss say tech healthcare rally
2797,Cramer week ahead Big week earn even bigger we...
2798,Cramer lightning round would Teradyne


against, be, of, a, in, to 등의 단어가 제거 된걸 확인 할 수 있다.

In [ ]:
 nltk.download('wordnet')
 nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

표제어 추출을 수행합니다. 
표제어 추출로 3인칭 단수 표현을 1인칭으로 바꾸고, 과거 현재형 동사를 현재형으로 바꿉니다.

In [ ]:
text['Headlines'] = text['Headlines'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
print(text.head(5))

                                           Headlines
0  [Cramer, :, Never, buy, stock, -, 'll, almost,...
1  [Cramer, :, I, help, investors, 2010, flash, c...
2  [Cramer, say, own, many, stock, little, cash, ...
3                  [Markets, lack, Christmas, cheer]
4  [S, &, P, tend, start, new, year, bullish, hap...


<ipython-input-11-9a6074ba03d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['Headlines'] = text['Headlines'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])


길이가 2이하인 단어에 대해서 제거하는 작업을 수행합니다.

In [ ]:
tokenized_doc = text['Headlines'].apply(lambda x: [word for word in x if len(word) > 2])
print(tokenized_doc[:5])

0    [Cramer, Never, buy, stock, 'll, almost, defin...
1    [Cramer, help, investors, 2010, flash, crash, ...
2    [Cramer, say, many, stock, little, cash, set, ...
3                    [Markets, lack, Christmas, cheer]
4       [S&P, tend, start, new, year, bullish, happen]
Name: Headlines, dtype: object


## TF-IDF 행렬 만들기

In [ ]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

# 다시 text['headline_text']에 재저장
text['Headlines'] = detokenized_doc

<ipython-input-31-22bea03db563>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['Headlines'] = detokenized_doc


In [ ]:
text['Headlines'][:5]

0    Cramer Never buy stock 'll almost definitely g...
1    Cramer help investors 2010 flash crash follow ...
2        Cramer say many stock little cash set failure
3                         Markets lack Christmas cheer
4               S&P tend start new year bullish happen
Name: Headlines, dtype: object

In [ ]:
# 상위 1,000개의 단어를 보존 
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000)
X = vectorizer.fit_transform(text['Headlines'])

In [ ]:
# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

TF-IDF 행렬의 크기 : (2800, 1000)


In [ ]:
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_top = lda_model.fit_transform(X)
print(lda_model.components_)
print(lda_model.components_.shape) 

[[0.18497468 0.17593632 0.15699984 ... 1.81773151 5.61382035 0.14518905]
 [0.14153605 0.14632257 0.14598271 ... 0.16301525 0.14744643 0.34320784]
 [0.14577237 0.16156511 0.14690868 ... 0.14717716 0.15730393 0.14682585]
 ...
 [0.15617438 0.14847824 1.41385013 ... 0.15323398 0.15372492 0.1445317 ]
 [0.16491956 0.14477343 0.93760316 ... 0.16068587 0.15795959 0.14738105]
 [0.15369156 1.36247822 0.1509845  ... 0.15995597 0.26845844 0.14331372]]
(10, 1000)


In [ ]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: [('cramer', 61.86), ('jim', 60.69), ('say', 47.65), ('earn', 36.96), ('market', 36.85)]
Topic 2: [('cramer', 32.62), ('stock', 24.36), ('round', 23.61), ('lightning', 23.19), ('jim', 23.09)]
Topic 3: [('money', 18.22), ('uber', 10.15), ('mad', 8.64), ('work', 7.09), ('management', 6.0)]
Topic 4: [('impact', 11.08), ('future', 10.45), ('best', 9.69), ('cramer', 9.56), ('start', 9.2)]
Topic 5: [('fed', 9.33), ('rate', 7.98), ('billion', 7.93), ('reopen', 7.89), ('disney', 7.76)]
Topic 6: [('street', 24.44), ('wall', 24.13), ('meat', 9.95), ('ipo', 8.96), ('know', 8.47)]
Topic 7: [('coronavirus', 24.16), ('say', 18.18), ('ceo', 16.26), ('cramer', 15.07), ('way', 13.02)]
Topic 8: [('round', 63.61), ('lightning', 63.4), ('cramer', 56.63), ('stock', 36.09), ('buy', 33.66)]
Topic 9: [('2020', 15.78), ('stay', 15.53), ('wells', 7.61), ('fargo', 7.6), ('pharma', 7.35)]
Topic 10: [('cramer', 29.27), ('round', 20.58), ('lightning', 20.48), ('stock', 17.96), ('want', 16.55)]


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


##새로운 TASK

In [ ]:
tokenized_doc[:5]

0    [Cramer, Never, buy, stock, 'll, almost, defin...
1    [Cramer, help, investors, 2010, flash, crash, ...
2    [Cramer, say, many, stock, little, cash, set, ...
3                    [Markets, lack, Christmas, cheer]
4       [S&P, tend, start, new, year, bullish, happen]
Name: Headlines, dtype: object

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(1, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1)]


In [ ]:
print(dictionary[66])

major


In [ ]:
len(dictionary)

4651

In [ ]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.100*"Cramer" + 0.045*"week" + 0.040*"earn" + 0.035*"ahead"')
(1, '0.067*"Cramer" + 0.030*"market" + 0.025*"Jim" + 0.022*"stock"')
(2, '0.044*"CEO" + 0.028*"say" + 0.024*"see" + 0.022*"Cramer"')
(3, '0.090*"Cramer" + 0.050*"stock" + 0.038*"buy" + 0.035*"Jim"')
(4, '0.030*"S&P" + 0.026*"500" + 0.020*"million" + 0.019*"Wells"')
(5, '0.079*"Cramer" + 0.060*"Jim" + 0.048*"say" + 0.044*"stock"')
(6, '0.060*"Street" + 0.057*"Wall" + 0.041*"Cramer" + 0.031*"Jim"')
(7, '0.160*"round" + 0.157*"lightning" + 0.156*"Cramer" + 0.038*"stock"')
(8, '0.028*"CEO" + 0.022*"best" + 0.021*"recovery" + 0.015*"Cramer"')
(9, '0.095*"Jim" + 0.092*"say" + 0.086*"Cramer" + 0.071*"include"')
(10, '0.058*"Cramer" + 0.037*"say" + 0.023*"CEO" + 0.020*"Jim"')
(11, '0.102*"Cramer" + 0.061*"stock" + 0.057*"Remix" + 0.035*"market"')
(12, '0.055*"S&P" + 0.049*"500" + 0.026*"Cramer" + 0.023*"Jim"')
(13, '0.051*"Cramer" + 0.031*"invest" + 0.024*"Remix" + 0.019*"Warren"')
(14, '0.037*"say" + 0.028*"CEO" + 0.020*"Cram

In [ ]:
print(ldamodel.print_topics())

[(0, '0.100*"Cramer" + 0.045*"week" + 0.040*"earn" + 0.035*"ahead" + 0.025*"trade" + 0.022*"Remix" + 0.019*"market" + 0.018*"report" + 0.013*"China" + 0.013*"stock"'), (1, '0.067*"Cramer" + 0.030*"market" + 0.025*"Jim" + 0.022*"stock" + 0.016*"game" + 0.014*"plan" + 0.014*"investors" + 0.013*"n\'t" + 0.012*"sell-off" + 0.012*"reveal"'), (2, '0.044*"CEO" + 0.028*"say" + 0.024*"see" + 0.022*"Cramer" + 0.018*"billion" + 0.018*"warn" + 0.015*"lot" + 0.014*"market" + 0.013*"\'ve" + 0.012*"\'We"'), (3, '0.090*"Cramer" + 0.050*"stock" + 0.038*"buy" + 0.035*"Jim" + 0.023*"say" + 0.013*"time" + 0.010*"n\'t" + 0.010*"worth" + 0.010*"still" + 0.009*"The"'), (4, '0.030*"S&P" + 0.026*"500" + 0.020*"million" + 0.019*"Fargo" + 0.019*"Wells" + 0.017*"drive" + 0.017*"future" + 0.014*"Chipotle" + 0.013*"surge" + 0.013*"since"'), (5, '0.079*"Cramer" + 0.060*"Jim" + 0.048*"say" + 0.044*"stock" + 0.027*"tech" + 0.020*"Salesforce" + 0.019*"company" + 0.010*"US-China" + 0.010*"play" + 0.010*"buy"'), (6, '0.0

시각화 하기

In [ ]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=4361d854fba38e2dc9133975d801fa2da253e1af18f106fd4528cb170653aeae
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=3995b146495a9cbbac046b9bce55ec4700dad7497b5f9beda465671122f182e4
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built pyLDAvis sklearn


In [ ]:
import pyLDAvis.gensim_models

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


문서 별 토픽 분포 보기

In [ ]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(4, 0.89444447)]
1 번째 문서의 topic 비율은 [(4, 0.90500003)]
2 번째 문서의 topic 비율은 [(5, 0.86428577)]
3 번째 문서의 topic 비율은 [(0, 0.010000001), (1, 0.010000001), (2, 0.010000001), (3, 0.010000001), (4, 0.010000001), (5, 0.010000001), (6, 0.010000001), (7, 0.010000001), (8, 0.010000001), (9, 0.010000001), (10, 0.21000001), (11, 0.61), (12, 0.010000001), (13, 0.010000001), (14, 0.010000001), (15, 0.010000001), (16, 0.010000001), (17, 0.010000001), (18, 0.010000001), (19, 0.010000001)]
4 번째 문서의 topic 비율은 [(12, 0.1690191), (13, 0.4063827), (14, 0.31834817)]


In [ ]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [ ]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,4.0,0.8944,"[(4, 0.89444447)]"
1,1,4.0,0.9050,"[(4, 0.90500003)]"
2,2,5.0,0.8643,"[(5, 0.86428577)]"
3,3,11.0,0.6100,"[(10, 0.21), (11, 0.61)]"
4,4,13.0,0.4064,"[(12, 0.16888294), (13, 0.40639022), (14, 0.31..."
5,5,10.0,0.3864,"[(0, 0.19410077), (4, 0.31327757), (10, 0.3863..."
6,6,0.0,0.5602,"[(0, 0.56023335), (10, 0.13849847), (13, 0.195..."
7,7,9.0,0.4779,"[(9, 0.47792253), (10, 0.4320775)]"
8,8,4.0,0.8417,"[(4, 0.8416667)]"
9,9,3.0,0.3390,"[(3, 0.33899438), (5, 0.2608911), (11, 0.30567..."
